# Full Name: Çağdaş Güven

# Student ID: 2738938

# Try to Tell Apples, Oranges and else apart

Images for this assignment are provided at github and they will be automatically downloaded right after imports. When the files are unzip there should be a folder:  ```AppleOrange```.
These are sample images, you can test with more if you want. While testing just for fun, I can try other stuff as well.

Your objective is to convert <font color="magenta"> **Apples to magenta** </font>, <font color="blue"> **Oranges to Blue** </font> , end blur everything else! While changing color, try to keep the original shading (i.e. try not provide a flat single color if possible, keep the shadings to the best you can).  

<font color="red">Do not be **too picky** and lose too much time</font>. Variety of images are provided so that you get the idea that generalizable / perfect filters are not easy to build. Yet your function is expected to work on more than one image at an acceptable level.  

By the same token, you can use the ```fakes```, also for for fun to see how your algorighm works on unrealted images, and why a general filter is not that easy...  

At the end as usual you are expected to **clear all outputs** and then save this file as **Week10_student_id.ipynb** and upload to the assignment at [ODTU Class](https://odtuclass2024f.metu.edu.tr/mod/assign/view.php?id=100364).  


# imports as usual
You are only allowed to use concepts related to what we have seen in class and use only the following imports. You can import sub-libraries with new names etc. but NO NEW LIBRARIES

In [ ]:
# not that all of them are necessary, but you are not allowed to import any new library
# yet as before, you can import sub libraries: i.e.:
#       from skimage import measure
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as pimg # check this out this is new
from numpy import cos, arccos, sin, pi, round
from numpy.linalg import matrix_rank as rank
from numpy.linalg import svd, eig
from scipy.linalg import orth
import cv2 as cv
from PIL import Image # good old pillow
import sklearn as skl # famous sci-kit learn
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin
from sklearn.utils import shuffle
from skimage.exposure import histogram , exposure
import skimage as ski # equally famous sci-kit image
!rm bug_numpy_utils.py 2>/dev/null  # at the first run file does not exits but error should print
!wget https://raw.githubusercontent.com/bugrakoku/bug_python_utils/main/bug_numpy_utils.py
from bug_numpy_utils import MatPrint, CData, text2mat # note that once these files are downloaded you can read their content.
!rm me536utils.py 2>/dev/null  # at the first run file does not exits but error should print
!wget https://raw.githubusercontent.com/bugrakoku/bug_python_utils/main/me536utils.py
from me536utils import RotMat

## get images


In [2]:
# !rm AppleOrange.zip 2>/dev/null # just in case
# !wget https://github.com/bugrakoku/data4all/raw/main/AppleOrange.zip # get the zip file
# !unzip AppleOrange.zip

# Get prepared
Below you can check images, perform tests runs, find critical color values etc.  
Leave all your preparation code so that I can see how you have reached the final implementation of the function. This part I will NOT run for evaluation! I will only run the ```AorO()``` and ```AorO2()``` functions in my tests!  

You can add as many code and text cells as you like below. But at the end, as I said above, I will just call the ```AorO()``` or ```AorO2()``` function.  
And please **DO CLEAR ALL OUTPUTS**!

Recall that you are not restricted to <font color="red">R</font>
<font color="green">G</font>
<font color="blue">B</font> at all!

# Part I: Manual threshold determination.
You can determine as many thresholds as you like.  


### Add explanations of what you do and why regarding the code(s) below

In [3]:
# your prep code


def plot_hsv_histogram(img_path):
    """
    this is a function to plot the histogram of the HSV channels of an image
    input:
        img_path: the path of the image
    output:
        None
        example:
        plot_hsv_histogram('img_path')
    
    created for learning purposes of opencv library and to play with images
    """
    # Load image
    img_read_BGR = cv.imread(img_path)
    img_read_RGB = cv.cvtColor(img_read_BGR, cv.COLOR_BGR2RGB)  # Convert BGR to RGB
    img_read_HSV = cv.cvtColor(img_read_BGR, cv.COLOR_BGR2HSV)  # Convert BGR to HSV

    # Separate the H, S, and V channels
    H, S, V = cv.split(img_read_HSV)

    # Plot histograms
    plt.figure(figsize=(16, 6))

    plt.subplot(1, 4, 1)
    plt.hist(H.ravel(), bins=180, range=[0, 179], color='orange', alpha=0.7)
    plt.title('Hue Histogram')
    plt.xlabel('Hue (0-179)')
    plt.ylabel('Frequency')

    plt.subplot(1, 4, 2)
    plt.hist(S.ravel(), bins=256, range=[0, 255], color='green', alpha=0.7)
    plt.title('Saturation Histogram')
    plt.xlabel('Saturation (0-255)')
    plt.ylabel('Frequency')

    plt.subplot(1, 4, 3)
    plt.hist(V.ravel(), bins=256, range=[0, 255], color='blue', alpha=0.7)
    plt.title('Value Histogram')
    plt.xlabel('Value (0-255)')
    plt.ylabel('Frequency')

    plt.subplot(1, 4, 4)
    plt.imshow(img_read_RGB)
    plt.title('Original Image')
    plt.axis('off')

    plt.tight_layout()
    plt.show()


def plot_RGB_histogram(image_path):
    """
    Reads an image from the given path and plots histograms for R, G, and B channels.
    Args:
        image_path: Path to the image file.

    created for if rgb channels are enough or HSV or other space is needed
    """
    # Load the image
    img_read_BGR = cv.imread(image_path)
    img_read_RGB = cv.cvtColor(img_read_BGR, cv.COLOR_BGR2RGB)  # Convert to RGB for consistent color channels

    # Split into R, G, B channels
    R, G, B = cv.split(img_read_RGB)

    # Plot histograms
    plt.figure(figsize=(16, 6))

    plt.subplot(1, 4, 1)
    plt.hist(R.ravel(), bins=256, range=[0, 256], color='r', alpha=0.7)
    plt.title('Red Channel Histogram')
    plt.xlabel('Intensity Value')
    plt.ylabel('Frequency')

    plt.subplot(1, 4, 2)
    plt.hist(G.ravel(), bins=256, range=[0, 256], color='g', alpha=0.7)
    plt.title('Green Channel Histogram')
    plt.xlabel('Intensity Value')
    plt.ylabel('Frequency')

    plt.subplot(1, 4, 3)
    plt.hist(B.ravel(), bins=256, range=[0, 256], color='b', alpha=0.7)
    plt.title('Blue Channel Histogram')
    plt.xlabel('Intensity Value')
    plt.ylabel('Frequency')

    plt.subplot(1, 4, 4)
    plt.imshow(img_read_RGB,cmap='gray')
    plt.title('Original Image')

    plt.tight_layout()
    plt.show()

def hsv_to_cv_ranges(h,  s, v):
    """
    Convert general HSV ranges to OpenCV-compatible ranges.

    Args:
        h_min (float): Minimum Hue (0-360).
        h_max (float): Maximum Hue (0-360).
        s_min (float): Minimum Saturation (0-100).
        s_max (float): Maximum Saturation (0-100).
        v_min (float): Minimum Value (0-100).
        v_max (float): Maximum Value (0-100).

    Returns:
        Tuple: Converted ranges in OpenCV format.
        (h_min_cv, h_max_cv, s_min_cv, s_max_cv, v_min_cv, v_max_cv)
    """
    # Convert H range (0-360) to OpenCV H range (0-180)
    h_cv = int((h / 360) * 180)

    # Convert S and V ranges (0-100) to OpenCV range (0-255)
    s_cv = int((s / 100) * 255)
    v_cv = int((v / 100) * 255)

    return h_cv, s_cv, v_cv




In [4]:
# potentially more code
#plot_hsv_histogram('AppleOrange/AorO1.jpg')


## ```AorO()``` function

In [5]:
'''
You are to properly implement the following function so that
Apples become magenta, orange blue in color, and rest is blurred where blurring is visually detectable.

'''


def AorO(img_path):
    """
    This function takes the path of an image, loads the image,
    converts the color of apples to magenta, oranges to blue, and blurs the rest.
    Shading (brightness) is preserved for apples and oranges.
    Returns the modified image.
    """
    # Input validation and image loading
    if not isinstance(img_path, str):
        print("Error: Image path must be a string.")
        return None

    if not img_path.lower().endswith((".jpg", ".jpeg", ".png", ".bmp", ".tiff")):
        print("Error: Invalid file extension. Supported extensions are .jpg, .jpeg, .png, .bmp, .tiff")
        return None

    img_read = cv.imread(img_path)
    if img_read is None:
        print(f"Error: Unable to load the image at '{img_path}'. The file may not exist or be corrupted.")
        return None

    if len(img_read.shape) != 3 or img_read.shape[2] != 3:
        print("Error: The input image is not a valid color image with three channels.")
        return None
    
    hsv_img = cv.cvtColor(img_read, cv.COLOR_BGR2HSV)

    # Define thresholds for apples and oranges in HSV
    apple_lower = np.array([2, 150, 0])    
    apple_upper = np.array([15, 255, 255])
    orange_lower = np.array([10, 150, 0])
    orange_upper = np.array([35, 255, 255])

    # Create masks for apples and oranges
    apple_mask = cv.inRange(hsv_img, apple_lower, apple_upper)
    orange_mask = cv.inRange(hsv_img, orange_lower, orange_upper)

    # Initialize result HSV image as a copy of the original
    result_hsv = hsv_img.copy()

    # Define HSV colors for magenta (apples) and blue (oranges)
    magenta_hsv = np.array([150, 255, 255], dtype=np.uint8)  # Hue: 150
    blue_hsv = np.array([105, 255, 255], dtype=np.uint8)     # Hue: 105

    # Apply Hue and Saturation changes for apple mask, keep original Value
    result_hsv[apple_mask > 0, 0] = magenta_hsv[0]  # Hue
    result_hsv[apple_mask > 0, 1] = magenta_hsv[1]  # Saturation

    # Apply Hue and Saturation changes for orange mask, keep original Value
    result_hsv[orange_mask > 0, 0] = blue_hsv[0]  # Hue
    result_hsv[orange_mask > 0, 1] = blue_hsv[1]  # Saturation

    # Convert HSV result back to BGR
    result_bgr = cv.cvtColor(result_hsv, cv.COLOR_HSV2BGR)

    # Blur the background
    blur_mask = cv.bitwise_not(cv.bitwise_or(apple_mask, orange_mask))
    blur = cv.GaussianBlur(img_read, (31, 31), 0)
    result_bgr[blur_mask > 0] = blur[blur_mask > 0]

    return result_bgr



# Part II: Use some clustering methods to find thresholds automatically
Assuming that there are apples and oranges in the image, analyze the colors in the image (such as histograms, or anything else you find fit), run some clustering algorithms on them and generate the image.

### Add explanations of what you do and why regarding the code(s) below

In [6]:
# your prep code

from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import numpy as np

def visualize_dominant_colors(img_path, num_clusters=3):
    """
    Extract and visualize dominant colors in an image using K-Means clustering.
    
    Args:
        img_path (str): Path to the input image.
        num_clusters (int): Number of color clusters.
    
    Returns:
        None: Displays the original image and a bar plot of dominant colors.
    """
    # Load image and convert to RGB
    img_bgr = cv.imread(img_path)
    img_rgb = cv.cvtColor(img_bgr, cv.COLOR_BGR2RGB)
    
    # Flatten the image
    img_flat = img_rgb.reshape((-1, 3))
    
    # Apply K-Means clustering
    kmeans = KMeans(n_clusters=num_clusters, random_state=42)
    kmeans.fit(img_flat)
    
    # Extract cluster centers (dominant colors) and labels
    dominant_colors = kmeans.cluster_centers_.astype(int)
    labels = kmeans.labels_
    
    # Count the pixels in each cluster
    label_counts = np.bincount(labels)
    
    # Sort clusters by pixel count
    sorted_indices = np.argsort(-label_counts)
    dominant_colors = dominant_colors[sorted_indices]
    label_counts = label_counts[sorted_indices]
    
    # Normalize counts to calculate proportions
    proportions = label_counts / sum(label_counts)
    
    # Plot the original image
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.imshow(img_rgb)
    plt.axis('off')
    plt.title("Original Image")
    
    # Plot the dominant colors
    plt.subplot(1, 2, 2)
    plt.bar(range(num_clusters), proportions, color=[dominant_colors[i] / 255 for i in range(num_clusters)])
    plt.title("Dominant Colors")
    plt.xlabel("Clusters")
    plt.ylabel("Proportion")
    plt.xticks(range(num_clusters), [f"Cluster {i+1}" for i in range(num_clusters)])
    
    plt.tight_layout()
    plt.show()

In [ ]:
# potentially more code
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import cv2 as cv
import numpy as np

# Read image
img = cv.imread('AppleOrange/AorO7.jpg')

# Convert from BGR to HSV
img_hsv = cv.cvtColor(img, cv.COLOR_BGR2HSV)

# Apply sharpening kernel
sharpen_kernel = np.array([[0, -1, 0], 
                            [-1, 5, -1], 
                            [0, -1, 0]])
sharpened_img = cv.filter2D(img_hsv, -1, sharpen_kernel)

# Apply Erosion after sharpening
kernel = np.ones((5, 5), np.uint8)  # Define a kernel for erosion
eroded_img = cv.erode(sharpened_img, kernel, iterations=1)

# Convert the eroded HSV image back to BGR for display
eroded_bgr = cv.cvtColor(eroded_img, cv.COLOR_HSV2BGR)

# Split the HSV channels from the eroded image
h, s, v = cv.split(eroded_img)

# Flatten the channels for visualization
h = h.flatten()
s = s.flatten()
v = v.flatten()

# Normalize the HSV values to ensure better visualization in 3D scatter
h = h / 180.0
s = s / 255.0
v = v / 255.0

# Plotting
fig = plt.figure(figsize=(15, 8))

# Subplot 1: Sharpened Image
ax1 = fig.add_subplot(1, 3, 1)
ax1.imshow(cv.cvtColor(cv.cvtColor(sharpened_img, cv.COLOR_HSV2BGR), cv.COLOR_BGR2RGB))
ax1.axis('off')
ax1.set_title('Sharpened Image')

# Subplot 2: Eroded Image Output
ax2 = fig.add_subplot(1, 3, 2)
ax2.imshow(cv.cvtColor(eroded_bgr, cv.COLOR_BGR2RGB))
ax2.axis('off')
ax2.set_title('Eroded Image')

# Subplot 3: 3D Scatter Plot
ax3 = fig.add_subplot(1, 3, 3, projection='3d')
scatter = ax3.scatter(h, s, v, c=np.array([h, s, v]).T, marker='o', s=1, alpha=0.6)
ax3.set_xlabel('Hue Channel')
ax3.set_ylabel('Saturation Channel')
ax3.set_zlabel('Value Channel')
ax3.set_title('3D HSV Scatter Plot (After Erosion)')

plt.tight_layout()
plt.show()



## ```AorO2()``` function

In [8]:
# '''
# You are to properly implement the following function so that
# Apples become magenta, orange blue in color, and rest is blurred where blurring is visually detectable.

# '''
# def AorO2(img_path):
#     """
#     This function applies a two-stage thresholding approach to segment apples and oranges
#     from the background. Apples are converted to magenta, oranges to blue, and the rest is blurred.

#     Args:
#         img_path (str): Path to the input image.

#     Returns:
#         result (ndarray): The modified image with segmentation applied.
#     """
#     # Load image
#     img_read = cv.imread(img_path)
#     if img_read is None:
#         raise ValueError(f"Image at path '{img_path}' could not be loaded.")

#     # Convert to HSV color space
#     hsv_img = cv.cvtColor(img_read, cv.COLOR_BGR2HSV)
#     h, s, v = cv.split(hsv_img)

#     # Stage 1: Global Thresholding (Suppress Background)
#     global_hue_lower, global_hue_upper = 0, 180  # Consider all hues
#     global_saturation_lower, global_saturation_upper = 50, 255  # Suppress low-saturation areas

#     global_mask = cv.inRange(hsv_img, 
#                              (global_hue_lower, global_saturation_lower, 0),
#                              (global_hue_upper, global_saturation_upper, 255))

#     # Apply global mask to filter the HSV image
#     hsv_filtered = cv.bitwise_and(hsv_img, hsv_img, mask=global_mask)

#     # Stage 2: Local Thresholding (Refine Foreground)
#     # Use Otsu's method on the saturation channel of the filtered image
#     _, local_thresh_saturation = cv.threshold(hsv_filtered[:, :, 1], 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)

#     # Combine masks
#     combined_mask = cv.bitwise_and(global_mask, local_thresh_saturation)

#     # Apply the combined mask
#     segmented_hsv = cv.bitwise_and(hsv_img, hsv_img, mask=combined_mask)

#     # Identify clusters: Apples (red hues) and Oranges (orange hues)
#     apple_lower = np.array([0, 30, 50])    
#     apple_upper = np.array([14, 255, 255])
#     orange_lower = np.array([14, 30, 50])
#     orange_upper = np.array([24, 255, 255])

#     apple_mask = cv.inRange(segmented_hsv, apple_lower, apple_upper)
#     orange_mask = cv.inRange(segmented_hsv, orange_lower, orange_upper)

#     # Create magenta (apples) and blue (oranges) in BGR
#     magenta = np.array([255, 0, 255], dtype=np.uint8)
#     blue = np.array([255, 0, 0], dtype=np.uint8)

#     # Initialize result
#     result = img_read.copy()

#     # Apply colors
#     result[apple_mask > 0] = magenta
#     result[orange_mask > 0] = blue

#     # Create blur mask (areas not identified as apple or orange)
#     blur_mask = cv.bitwise_not(cv.bitwise_or(apple_mask, orange_mask))

#     # Apply blur to remaining regions
#     blur = cv.GaussianBlur(img_read, (31, 31), 0)
#     result[blur_mask > 0] = blur[blur_mask > 0]

#     return result

In [9]:


def AorO2(img_path):
    """
    AorO2 using K-Means clustering with dynamic cluster assignment and HSV shading preservation.
    
    Args:
        img_path (str): Path to the input image.
        num_clusters (int): Number of clusters for K-Means.
        debug (bool): Enable debug mode for visualizing clusters.(removed for submission)
    
    Returns:
        result (ndarray): Processed image with apples, oranges, and background.
    """
    # Input validation and image loading
    if not isinstance(img_path, str):
        print("Error: Image path must be a string.")
        return None

    if not img_path.lower().endswith((".jpg", ".jpeg", ".png", ".bmp", ".tiff")):
        print("Error: Invalid file extension. Supported extensions are .jpg, .jpeg, .png, .bmp, .tiff")
        return None

    img_read = cv.imread(img_path)
    if img_read is None:
        print(f"Error: Unable to load the image at '{img_path}'. The file may not exist or be corrupted.")
        return None

    if len(img_read.shape) != 3 or img_read.shape[2] != 3:
        print("Error: The input image is not a valid color image with three channels.")
        return None

    num_clusters = 10
    

    # Preprocess: Sharpen the image
    sharpen_kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
    img_sharpened = cv.filter2D(img_read, -1, sharpen_kernel)

    # Convert to HSV color space
    hsv_img = cv.cvtColor(img_sharpened, cv.COLOR_BGR2HSV)
    h, w, _ = hsv_img.shape

    # Flatten HSV for clustering
    flat_hsv = hsv_img.reshape((-1, 3))

    # Perform K-Means clustering
    kmeans = KMeans(n_clusters=num_clusters, random_state=42, max_iter=300)
    kmeans.fit(flat_hsv)
    labels = kmeans.labels_
    centers = kmeans.cluster_centers_


    # Reshape labels to the original image dimensions
    cluster_map = labels.reshape((h, w))

    # Initialize masks
    apple_mask = np.zeros((h, w), dtype=np.uint8)
    orange_mask = np.zeros((h, w), dtype=np.uint8)
    background_mask = np.zeros((h, w), dtype=np.uint8)

    # Dynamically assign clusters based on center values
    for i, center in enumerate(centers):
        hue, sat, val = center
        if 10 <= hue <= 35 and sat > 200 :  # Likely orange
            orange_mask[cluster_map == i] = 255
        elif (2 <= hue <= 15 or 170 <= hue <= 179) and 100 <= sat <= 200 :  # Likely apple
            apple_mask[cluster_map == i] = 255
        else:  # Likely background
            background_mask[cluster_map == i] = 255

    # Refine masks with morphological operations
    kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, (5, 5))
    apple_mask = cv.dilate(apple_mask, kernel, iterations=2)
    orange_mask = cv.dilate(orange_mask, kernel, iterations=2)

    # Create HSV colors for magenta (apples) and blue (oranges)
    magenta_hsv = np.array([150, 255, 255], dtype=np.uint8)
    blue_hsv = np.array([105, 255, 255], dtype=np.uint8)

    # Apply HSV changes to the original image
    result_hsv = hsv_img.copy()

    # Apply Hue and Saturation for apple mask, keep original Value
    result_hsv[apple_mask > 0, 0] = magenta_hsv[0]  # Hue
    result_hsv[apple_mask > 0, 1] = magenta_hsv[1]  # Saturation

    # Apply Hue and Saturation for orange mask, keep original Value
    result_hsv[orange_mask > 0, 0] = blue_hsv[0]  # Hue
    result_hsv[orange_mask > 0, 1] = blue_hsv[1]  # Saturation

    # Convert HSV back to BGR for final result
    result_bgr = cv.cvtColor(result_hsv, cv.COLOR_HSV2BGR)

    # Blur the background
    blur_mask = cv.bitwise_not(cv.bitwise_or(apple_mask, orange_mask))
    blur = cv.GaussianBlur(img_read, (31, 31), 0)
    result_bgr[blur_mask > 0] = blur[blur_mask > 0]



    return result_bgr




### Explanation of the Function

The `AorO2` function processes an image to:
1. Identify and highlight apples and oranges using specific colors (magenta and blue).
2. Blur the background for aesthetic distinction.
3. Use K-Means clustering and HSV color space for dynamic and robust color-based segmentation.

### Step-by-Step Breakdown

1. **Input Validation**:
   - Checks if the input path is a string.
   - Ensures the file extension is valid (`.jpg`, `.jpeg`, `.png`, `.bmp`, `.tiff`).
   - Attempts to load the image using OpenCV (`cv.imread`).
   - Validates that the loaded image is a 3-channel color image.

2. **Preprocessing**:
   - A sharpening filter (`sharpen_kernel`) is applied to the image to enhance edges and details before processing.

3. **Color Space Conversion**:
   - Converts the sharpened image from BGR (Blue-Green-Red) to HSV (Hue-Saturation-Value) format for better color differentiation.

4. **Data Preparation for Clustering**:
   - Flattens the HSV image into a 2D array (`flat_hsv`) where each pixel's HSV values are treated as a data point.

5. **K-Means Clustering**:
   - Performs clustering on the flattened data to group pixels into `num_clusters` (default: 10) based on their color properties.
   - Outputs:
     - `labels`: Cluster assignments for each pixel.
     - `centers`: HSV values representing the cluster centers.

6. **Cluster Assignment**:
   - Reshapes the cluster labels back into the original image dimensions (`cluster_map`).
   - Dynamically assigns clusters to apples, oranges, and the background based on heuristics:
     - **Apples**: Hue between 2-15 or 170-179, medium saturation (100-200).
     - **Oranges**: Hue between 10-35, high saturation (>200).
     - **Background**: Anything not classified as apple or orange.

7. **Refinement**:
   - Applies morphological dilation using an elliptical kernel to clean up and fill small gaps in the masks.

8. **Color Application**:
   - Converts masks into specific HSV colors:
     - **Apples**: Magenta (`Hue=150`).
     - **Oranges**: Blue (`Hue=105`).
   - Preserves the original Value channel (shading/brightness) to maintain the realistic appearance of the fruits.

9. **Reconstruction**:
   - Converts the processed HSV image (`result_hsv`) back to BGR format for visualization.

10. **Background Blurring**:
    - Creates a blur mask for areas not identified as apples or oranges.
    - Applies a Gaussian blur to those areas for visual emphasis.

11. **Return**:
    - Returns the final processed image (`result_bgr`) with:
      - Apples in magenta.
      - Oranges in blue.
      - Blurred background.



# My test will be performed below
Using my own code either at the end of this file or separately on my computer, I will just call the  ```AorO()``` or ```AorO2() functions in different ways and enjoy the outcomes :)